# Use the my-first-ds datasource

In [1]:
# Install the DagsHub client
%pip install -q dagshub


Note: you may need to restart the kernel to use updated packages.


In [2]:
from dagshub.data_engine import datasources

ds = datasources.get("rebornrulz/Rulz-AI", "my-first-ds")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=0dbb49b5-429a-467e-afc2-fda391fa50b0&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=4ec85ba86c9296a7867febb925d0b014e06a80c91a6a6a999ab1c9bee413d5d2




## Query your data
Datasources can be filtered using a pandas-like syntax.

[For the full list of possible filters, see the docs](https://dagshub.com/docs/use_cases/data_engine/query_and_create_subsets).

In [ ]:
# Example for simple query: get all data points with size larger than 1 byte
newQuery = ds["size"] > 1

# Display query results as data frame
newQuery.all().dataframe

,path,datapoint_id,dagshub_download_url,size
1,file0.png,41925882,https://dagshub.com/api/v1/repos/user/reponame...,116656
2,file1.jpeg,41925883,https://dagshub.com/api/v1/repos/user/reponame...,142949
3,file2.png,41925884,https://dagshub.com/api/v1/repos/user/reponame...,2982592
0,file3.gif,41925881,https://dagshub.com/api/v1/repos/user/reponame...,569995


## Additional Data Engine Capabilities

Data Engine capabilities also includes: [data enrichments](https://dagshub.com/docs/use_cases/data_engine/enrich_datasource/), [visualizations](https://dagshub.com/docs/use_cases/data_engine/visualizing_datasets/), [annotations](https://dagshub.com/docs/use_cases/data_engine/annotate_data/), [creating datasets](https://dagshub.com/docs/use_cases/data_engine/query_and_create_subsets/) and [model training](https://dagshub.com/docs/use_cases/data_engine/train_model/).